In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import sqlite3
import re
from tqdm import tqdm
import json
import csv
#from query import Query

In [3]:
def count_lines(fname):
    with open(fname) as f:
        return sum(1 for line in f)


def detokenize(tokens):
    ret = ''
    for g, a in zip(tokens['gloss'], tokens['after']):
        ret += g + a
    return ret.strip()

In [4]:
from collections import defaultdict
from copy import deepcopy
import re


re_whitespace = re.compile(r'\s+', flags=re.UNICODE)


class Query:

    agg_ops = ['', 'MAX', 'MIN', 'COUNT', 'SUM', 'AVG']
    cond_ops = ['=', '>', '<', 'OP']
    syms = ['SELECT', 'WHERE', 'AND', 'COL', 'TABLE', 'CAPTION', 'PAGE', 'SECTION', 'OP', 'COND', 'QUESTION', 'AGG', 'AGGOPS', 'CONDOPS']

    def __init__(self, sel_index, agg_index, conditions=tuple(), ordered=False):
        self.sel_index = sel_index
        self.agg_index = agg_index
        self.conditions = list(conditions)
        self.ordered = ordered

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            indices = self.sel_index == other.sel_index and self.agg_index == other.agg_index
            if other.ordered:
                conds = [(col, op, str(cond).lower()) for col, op, cond in self.conditions] == [(col, op, str(cond).lower()) for col, op, cond in other.conditions]
            else:
                conds = set([(col, op, str(cond).lower()) for col, op, cond in self.conditions]) == set([(col, op, str(cond).lower()) for col, op, cond in other.conditions])

            return indices and conds
        return NotImplemented

    def __ne__(self, other):
        if isinstance(other, self.__class__):
            return not self.__eq__(other)
        return NotImplemented

    def __hash__(self):
        return hash(tuple(sorted(self.__dict__.items())))

    def __repr__(self):
        rep = 'SELECT {agg} {sel} FROM table'.format(
            agg=self.agg_ops[self.agg_index],
            sel='col{}'.format(self.sel_index),
        )
        if self.conditions:
            rep +=  ' WHERE ' + ' AND '.join(['{} {} {}'.format('col{}'.format(i), self.cond_ops[o],'"{}"'.format(v) if not str(v).isnumeric() else v) for i, o, v in self.conditions])
        return rep

    def to_dict(self):
        return {'sel': self.sel_index, 'agg': self.agg_index, 'conds': self.conditions}

    def lower(self):
        conds = []
        for col, op, cond in self.conditions:
            conds.append([col, op, cond.lower()])
        return self.__class__(self.sel_index, self.agg_index, conds)

    @classmethod
    def from_dict(cls, d, ordered=False):
        return cls(sel_index=d['sel'], agg_index=d['agg'], conditions=d['conds'], ordered=ordered)

    @classmethod
    def from_tokenized_dict(cls, d):
        conds = []
        for col, op, val in d['conds']:
            conds.append([col, op, detokenize(val)])
        return cls(d['sel'], d['agg'], conds)

    @classmethod
    def from_generated_dict(cls, d):
        conds = []
        for col, op, val in d['conds']:
            end = len(val['words'])
            conds.append([col, op, detokenize(val)])
        return cls(d['sel'], d['agg'], conds)

    @classmethod
    def from_sequence(cls, sequence, table, lowercase=True):
        sequence = deepcopy(sequence)
        if 'symend' in sequence['words']:
            end = sequence['words'].index('symend')
            for k, v in sequence.items():
                sequence[k] = v[:end]
        terms = [{'gloss': g, 'word': w, 'after': a} for  g, w, a in zip(sequence['gloss'], sequence['words'], sequence['after'])]
        headers = [detokenize(h) for h in table['header']]

        # lowercase everything and truncate sequence
        if lowercase:
            headers = [h.lower() for h in headers]
            for i, t in enumerate(terms):
                for k, v in t.items():
                    t[k] = v.lower()
        headers_no_whitespcae = [re.sub(re_whitespace, '', h) for h in headers]

        # get select
        if 'symselect' != terms.pop(0)['word']:
            raise Exception('Missing symselect operator')

        # get aggregation
        if 'symagg' != terms.pop(0)['word']:
            raise Exception('Missing symagg operator')
        agg_op = terms.pop(0)['word']

        if agg_op == 'symcol':
            agg_op = ''
        else:
            if 'symcol' != terms.pop(0)['word']:
                raise Exception('Missing aggregation column')
        try:
            agg_op = cls.agg_ops.index(agg_op.upper())
        except Exception as e:
            raise Exception('Invalid agg op {}'.format(agg_op))
        
        def find_column(name):
            return headers_no_whitespcae.index(re.sub(re_whitespace, '', name))

        def flatten(tokens):
            ret = {'words': [], 'after': [], 'gloss': []}
            for t in tokens:
                ret['words'].append(t['word'])
                ret['after'].append(t['after'])
                ret['gloss'].append(t['gloss'])
            return ret
        where_index = [i for i, t in enumerate(terms) if t['word'] == 'symwhere']
        where_index = where_index[0] if where_index else len(terms)
        flat = flatten(terms[:where_index])
        try:
            agg_col = find_column(detokenize(flat))
        except Exception as e:
            raise Exception('Cannot find aggregation column {}'.format(flat['words']))
        where_terms = terms[where_index+1:]

        # get conditions
        conditions = []
        while where_terms:
            t = where_terms.pop(0)
            flat = flatten(where_terms)
            if t['word'] != 'symcol':
                raise Exception('Missing conditional column {}'.format(flat['words']))
            try:
                op_index = flat['words'].index('symop')
                col_tokens = flatten(where_terms[:op_index])
            except Exception as e:
                raise Exception('Missing conditional operator {}'.format(flat['words']))
            cond_op = where_terms[op_index+1]['word']
            try:
                cond_op = cls.cond_ops.index(cond_op.upper())
            except Exception as e:
                raise Exception('Invalid cond op {}'.format(cond_op))
            try:
                cond_col = find_column(detokenize(col_tokens))
            except Exception as e:
                raise Exception('Cannot find conditional column {}'.format(col_tokens['words']))
            try:
                val_index = flat['words'].index('symcond')
            except Exception as e:
                raise Exception('Cannot find conditional value {}'.format(flat['words']))

            where_terms = where_terms[val_index+1:]
            flat = flatten(where_terms)
            val_end_index = flat['words'].index('symand') if 'symand' in flat['words'] else len(where_terms)
            cond_val = detokenize(flatten(where_terms[:val_end_index]))
            conditions.append([cond_col, cond_op, cond_val])
            where_terms = where_terms[val_end_index+1:]
        q = cls(agg_col, agg_op, conditions)
        return q

    @classmethod
    def from_partial_sequence(cls, agg_col, agg_op, sequence, table, lowercase=True):
        sequence = deepcopy(sequence)
        if 'symend' in sequence['words']:
            end = sequence['words'].index('symend')
            for k, v in sequence.items():
                sequence[k] = v[:end]
        terms = [{'gloss': g, 'word': w, 'after': a} for  g, w, a in zip(sequence['gloss'], sequence['words'], sequence['after'])]
        headers = [detokenize(h) for h in table['header']]

        # lowercase everything and truncate sequence
        if lowercase:
            headers = [h.lower() for h in headers]
            for i, t in enumerate(terms):
                for k, v in t.items():
                    t[k] = v.lower()
        headers_no_whitespcae = [re.sub(re_whitespace, '', h) for h in headers]

        def find_column(name):
            return headers_no_whitespcae.index(re.sub(re_whitespace, '', name))

        def flatten(tokens):
            ret = {'words': [], 'after': [], 'gloss': []}
            for t in tokens:
                ret['words'].append(t['word'])
                ret['after'].append(t['after'])
                ret['gloss'].append(t['gloss'])
            return ret
        where_index = [i for i, t in enumerate(terms) if t['word'] == 'symwhere']
        where_index = where_index[0] if where_index else len(terms)
        where_terms = terms[where_index+1:]

        # get conditions
        conditions = []
        while where_terms:
            t = where_terms.pop(0)
            flat = flatten(where_terms)
            if t['word'] != 'symcol':
                raise Exception('Missing conditional column {}'.format(flat['words']))
            try:
                op_index = flat['words'].index('symop')
                col_tokens = flatten(where_terms[:op_index])
            except Exception as e:
                raise Exception('Missing conditional operator {}'.format(flat['words']))
            cond_op = where_terms[op_index+1]['word']
            try:
                cond_op = cls.cond_ops.index(cond_op.upper())
            except Exception as e:
                raise Exception('Invalid cond op {}'.format(cond_op))
            try:
                cond_col = find_column(detokenize(col_tokens))
            except Exception as e:
                raise Exception('Cannot find conditional column {}'.format(col_tokens['words']))
            try:
                val_index = flat['words'].index('symcond')
            except Exception as e:
                raise Exception('Cannot find conditional value {}'.format(flat['words']))

            where_terms = where_terms[val_index+1:]
            flat = flatten(where_terms)
            val_end_index = flat['words'].index('symand') if 'symand' in flat['words'] else len(where_terms)
            cond_val = detokenize(flatten(where_terms[:val_end_index]))
            conditions.append([cond_col, cond_op, cond_val])
            where_terms = where_terms[val_end_index+1:]
        q = cls(agg_col, agg_op, conditions)
        return q

In [5]:
# Open the input file
with open('train.jsonl', 'r') as input_file:

    # Open the output file
    with open('train_clean.csv', 'w', newline='') as output_file:

        # Create a CSV writer
        writer = csv.writer(output_file)

        # Write the header row
        writer.writerow(['phase', 'table_id', 'question', 'sel', 'conds', 'agg'])

        # Loop over each line in the input file
        for line in input_file:

            # Parse the JSON data
            data = json.loads(line)

            # Extract the relevant fields
            phase = data['phase']
            table_id = data['table_id']
            question = data['question']
            sel = data['sql']['sel']
            conds = data['sql']['conds']
            agg = data['sql']['agg']

            # Write the data to the output file
            writer.writerow([phase, table_id, question, sel, conds, agg])


FileNotFoundError: [Errno 2] No such file or directory: 'train.jsonl'

In [6]:
train_data_csv = pd.read_csv('train.csv')
train_data_csv.head()

,phase,table_id,question,sel,conds,agg
0,1,1-1000181-1,Tell me what the notes are for South Australia,5,"[[3, 0, 'SOUTH AUSTRALIA']]",0
1,1,1-1000181-1,What is the current series where the new serie...,4,"[[5, 0, 'New series began in June 2011']]",0
2,1,1-1000181-1,What is the format for South Australia?,2,"[[0, 0, 'South Australia']]",0
3,1,1-1000181-1,Name the background colour for the Australian ...,1,"[[0, 0, 'Australian Capital Territory']]",0
4,1,1-10007452-3,how many times is the fuel propulsion is cng?,3,"[[5, 0, 'CNG']]",3


In [9]:
all_query=[]
n_lines = 0
with open('train.jsonl') as f:
    for l in f:
        n_lines += 1
with open('train.jsonl') as f:
    for l in tqdm(f, total=n_lines):
        d = json.loads(l)
        query = Query.from_dict(d['sql'])
        all_query.append(query)
        

100%|██████████| 56355/56355 [00:00<00:00, 70910.20it/s] 


In [11]:
train_data_csv.head()

,phase,table_id,question,sel,conds,agg,sql
0,1,1-1000181-1,Tell me what the notes are for South Australia,5,"[[3, 0, 'SOUTH AUSTRALIA']]",0,"SELECT col5 FROM table WHERE col3 = ""SOUTH AU..."
1,1,1-1000181-1,What is the current series where the new serie...,4,"[[5, 0, 'New series began in June 2011']]",0,"SELECT col4 FROM table WHERE col5 = ""New seri..."
2,1,1-1000181-1,What is the format for South Australia?,2,"[[0, 0, 'South Australia']]",0,"SELECT col2 FROM table WHERE col0 = ""South Au..."
3,1,1-1000181-1,Name the background colour for the Australian ...,1,"[[0, 0, 'Australian Capital Territory']]",0,"SELECT col1 FROM table WHERE col0 = ""Australi..."
4,1,1-10007452-3,how many times is the fuel propulsion is cng?,3,"[[5, 0, 'CNG']]",3,"SELECT COUNT col3 FROM table WHERE col5 = ""CNG"""


In [10]:
train_data_csv['sql'] = all_query

In [62]:
train_data_csv.to_csv('wikisql.csv', index=False)

In [12]:
all_query[0:5]

[SELECT  col5 FROM table WHERE col3 = "SOUTH AUSTRALIA",
 SELECT  col4 FROM table WHERE col5 = "New series began in June 2011",
 SELECT  col2 FROM table WHERE col0 = "South Australia",
 SELECT  col1 FROM table WHERE col0 = "Australian Capital Territory",
 SELECT COUNT col3 FROM table WHERE col5 = "CNG"]

In [26]:
#Reading it again after saving it for further use
wikisql = pd.read_csv('wikisql.csv')
wikisql.head()

,phase,table_id,question,sel,conds,agg,sql
0,1,1-1000181-1,Tell me what the notes are for South Australia,5,"[[3, 0, 'SOUTH AUSTRALIA']]",0,"SELECT col5 FROM table WHERE col3 = ""SOUTH AU..."
1,1,1-1000181-1,What is the current series where the new serie...,4,"[[5, 0, 'New series began in June 2011']]",0,"SELECT col4 FROM table WHERE col5 = ""New seri..."
2,1,1-1000181-1,What is the format for South Australia?,2,"[[0, 0, 'South Australia']]",0,"SELECT col2 FROM table WHERE col0 = ""South Au..."
3,1,1-1000181-1,Name the background colour for the Australian ...,1,"[[0, 0, 'Australian Capital Territory']]",0,"SELECT col1 FROM table WHERE col0 = ""Australi..."
4,1,1-10007452-3,how many times is the fuel propulsion is cng?,3,"[[5, 0, 'CNG']]",3,"SELECT COUNT col3 FROM table WHERE col5 = ""CNG"""


In [27]:
#checking how the json file looks
with open('train.jsonl') as f:
    # read the first line and print it
    first_line = f.readline()
    print(first_line)

    # get the total number of lines in the file
    n_lines = sum(1 for line in f)

    # go back to the beginning of the file
    f.seek(0)

    # loop over each line in the file
    for l in tqdm(f, total=n_lines):
        d = json.loads(l)
        

{"phase": 1, "table_id": "1-1000181-1", "question": "Tell me what the notes are for South Australia ", "sql": {"sel": 5, "conds": [[3, 0, "SOUTH AUSTRALIA"]], "agg": 0}}



56355it [00:00, 188304.22it/s]                           


In [23]:
# Open the input file
with open('train.jsonl', 'r') as input_file:

    # Open the output file
    with open('output.csv', 'w', newline='') as output_file:

        # Create a CSV writer
        writer = csv.writer(output_file)

        # Write the header row
        writer.writerow(['db_id', 'TQL', 'SQL', 'dataset', 'fileName', 'filePath', 'result'])

        # Loop over each line in the input file
        for line in input_file:

            # Parse the JSON data
            data = json.loads(line)

            # Extract the relevant fields
            db_id = data['table_id']  
            TQL = data['question']
            SQL = Query.from_dict(data['sql'])
            dataset = 'train'  # set your dataset here
            fileName = f'{db_id}.sqlite'
            filePath = f'sqliteDB/{fileName}'
            result = None  

            # Write the data to the output file
            writer.writerow([db_id, TQL, SQL, dataset, fileName, filePath, result])

In [24]:
output1 = pd.read_csv('output.csv')
output1.head()

,db_id,TQL,SQL,dataset,fileName,filePath,result
0,1-1000181-1,Tell me what the notes are for South Australia,"SELECT col5 FROM table WHERE col3 = ""SOUTH AU...",train,1-1000181-1.sqlite,sqliteDB/1-1000181-1.sqlite,NaN
1,1-1000181-1,What is the current series where the new serie...,"SELECT col4 FROM table WHERE col5 = ""New seri...",train,1-1000181-1.sqlite,sqliteDB/1-1000181-1.sqlite,NaN
2,1-1000181-1,What is the format for South Australia?,"SELECT col2 FROM table WHERE col0 = ""South Au...",train,1-1000181-1.sqlite,sqliteDB/1-1000181-1.sqlite,NaN
3,1-1000181-1,Name the background colour for the Australian ...,"SELECT col1 FROM table WHERE col0 = ""Australi...",train,1-1000181-1.sqlite,sqliteDB/1-1000181-1.sqlite,NaN
4,1-10007452-3,how many times is the fuel propulsion is cng?,"SELECT COUNT col3 FROM table WHERE col5 = ""CNG""",train,1-10007452-3.sqlite,sqliteDB/1-10007452-3.sqlite,NaN


In [ ]:
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE my_table (
                db_id TEXT,
                TQL TEXT,
                SQL TEXT,
                dataset TEXT,
                fileName TEXT,
                filePath TEXT,
                result TEXT
            )''')

# Insert data into table
for i, row in df.iterrows():
    db_id = row['db_id']
    tql = row['TQL']
    sql = row['SQL']
    dataset = row['dataset']
    fileName = row['fileName']
    filePath = row['filePath']
    result = row['result']
    
    # Execute the query and insert the result into the database
    try:
        res_dict = pd.read_sql_query(sql, conn).to_dict()
        c.execute(f"INSERT INTO my_table VALUES ('{db_id}', '{tql}', '{sql}', '{dataset}', '{fileName}', '{filePath}', '{res_dict}')")
        conn.commit()
    except Exception as err:
        print(f"Error executing query for {db_id}: {err}")

# Save changes to database
conn.commit()

# Close database connection
conn.close()

# Run queries and store results
results = []
conn = sqlite3.connect('database.db')
c = conn.cursor()

for row in c.execute("SELECT SQL, filePath FROM my_table"):
    query = row[0]
    file_path = row[1]
    
    # Connect to SQLite database
    conn2 = sqlite3.connect(file_path)
    c2 = conn2.cursor()
    
    try:
        # Execute query
        res = pd.read_sql_query(query, conn2).to_dict()
        
        # Append result to results list
        results.append(res)
        
        # Update result in database
        c.execute(f"UPDATE my_table SET result = '{res}' WHERE SQL = '{query}'")
        
        # Save changes to database
        conn.commit()
        
    except:
        # If query fails, append None to results list
        results.append(None)

    # Close database connection
    conn2.close()

# Close database connection
conn.close()

In [ ]:
import json
import sqlite3

# specify the location of the WikiSQL dataset JSON files
train_data_file = "train.jsonl"
dev_data_file = "dev.jsonl"
test_data_file = "test.jsonl"

# create a SQLite database to store the tables
conn = sqlite3.connect("wikisql.db")
cursor = conn.cursor()

# iterate over each data file
for data_file in [train_data_file, dev_data_file, test_data_file]:
    with open(data_file, "r") as f:
        # iterate over each example in the data file
        for line in f:
            example = json.loads(line.strip())
            table_id = example["table_id"]
            table_data = example["table"]
            column_names = table_data["header"]
            rows = table_data["rows"]
            
            # create the SQL query to create the table
            create_table_query = f"CREATE TABLE {table_id} ("
            for i, column_name in enumerate(column_names):
                create_table_query += f"{column_name} TEXT"
                if i < len(column_names) - 1:
                    create_table_query += ","
            create_table_query += ")"
            
            # execute the SQL query to create the table
            cursor.execute(create_table_query)
            
            # create the SQL query to insert the rows into the table
            insert_rows_query = f"INSERT INTO {table_id} VALUES "
            for i, row in enumerate(rows):
                insert_rows_query += "("
                for j, value in enumerate(row):
                    insert_rows_query += f"'{value}'"
                    if j < len(row) - 1:
                        insert_rows_query += ","
                insert_rows_query += ")"
                if i < len(rows) - 1:
                    insert_rows_query += ","
                    
            # execute the SQL query to insert the rows into the table
            cursor.execute(insert_rows_query)
            
# commit the changes to the database and close the connection
conn.commit()
conn.close()


In [17]:
# json to desired query data format:
# Open the input file
with open('train.jsonl', 'r') as input_file:

    # Open the output file
    with open('wikisql_querydata.csv', 'w', newline='') as output_file:

        # Create a CSV writer
        writer = csv.writer(output_file)

        # Write the header row
        writer.writerow(['db_id', 'TQL', 'SQL', 'dataset', 'fileName', 'filePath', 'result'])

        # Loop over each line in the input file
        for line in input_file:

            # Parse the JSON data
            data = json.loads(line)

            # Extract the relevant fields
            #db_id = data['db_id']
            #TQL = data['question']
            #SQL = data['sql']['sql']
            #dataset = 'train'
            #fileName = f'{data["db_id"]}.sqlite'
            #filePath = f'sqliteDB/{fileName}'
            #result = None
            
            # Write the data to the output file
            #writer.writerow([db_id, TQL, SQL, dataset, fileName, filePath, result])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
connection = sqlite3.connect(file_path)
cursor = connection.cursor()
cursor.execute(sql_query)
result = cursor.fetchall()

In [ ]:
# Define the headers for the CSV file
headers = ['db_id', 'Text', 'SQL', 'dataset', 'fileName', 'filePath', 'result']

# Create an empty list to store the dictionaries
csv_data = []

# Loop through the train data
for i in range(len(train)):
    # Extract the required information from the dictionary
    phase = train[i]['phase']
    question = train[i]['question']
    sql_query = train[i]['sql']['human_readable']
    dataset = train[i]['table']['name']
    file_name = f"{dataset}.sqlite"
    file_path = f"sqliteDB/{file_name}"
    
    # Connect to the database
    conn = sqlite3.connect(file_path)
    
    # Create a cursor object
    cur = conn.cursor()
    
    # Execute the SQL query
    cur.execute(sql_query)
    
    # Fetch the result
    result = cur.fetchall()
    
    # Close the database connection
    conn.close()
    
    # Create a dictionary with the required information
    csv_dict = {'db_id': phase, 'Text': question, 'SQL': sql_query, 'dataset': dataset,
                'fileName': file_name, 'filePath': file_path, 'result': result}
    
    # Append the dictionary to the list
    csv_data.append(csv_dict)

# Write the list of dictionaries to a CSV file
with open('output.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    writer.writerows(csv_data)